# Exercise 5: Planner pattern implementation

In [1]:
from dotenv import load_dotenv
_ = load_dotenv()

In [3]:
import operator
from typing import Annotated, List, TypedDict, Union
from langgraph.graph import StateGraph, END, START
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

# --- Setup LLM ---
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# --- 1. Define the State ---
class PlanState(TypedDict):
    request: str                # The user's original request
    plan: List[str]             # The queue of remaining steps
    past_steps: List[str]       # Steps we have already finished
    results: List[str]          # The data collected from executed steps

# --- 2. Define the PROMPTS ---

PLANNER_PROMPT = """
You are a Project Manager. 
Break down the following user request into 3 distinct, actionable steps.
Return ONLY the steps as a Python list of strings. Do not add numbering or extra text.

Example Output:
["Search for X", "Analyze Y", "Write Z"]

User Request: {request}
"""

EXECUTOR_PROMPT = """
You are a Specialist Worker.
Execute the following step: "{step}"
Context from previous steps: {context}

(Note: Since I cannot browse the real internet, simulate the research by generating realistic, detailed information about this topic.)
"""

REPORTER_PROMPT = """
You are a Final Reporter.
Read the following research results and compile a final answer to the original request.

Original Request: {request}
Results:
{results}
"""

# --- 3. Define the Nodes ---

def planner_node(state: PlanState):
    print(f"\n--- PLANNER: Analyzing request '{state['request']}' ---")
    
    messages = [
        SystemMessage(content=PLANNER_PROMPT.format(request=state['request']))
    ]
    
    response = llm.invoke(messages)
    
    # Simple parsing to turn the string representation of a list into an actual list
    # (In production, use structured outputs/Pydantic)
    try:
        # This is a hacky way to parse "[...]" string to list for this demo
        import ast
        plan = ast.literal_eval(response.content)
    except:
        # Fallback if LLM messes up format
        plan = response.content.split("\n")

    print(f"--- PLANNER: Created roadmap: {plan} ---")
    
    return {"plan": plan, "results": [], "past_steps": []}

def executor_node(state: PlanState):
    # Get the next step
    plan = state["plan"]
    current_step = plan[0]
    
    print(f"--- EXECUTOR: Working on step: '{current_step}' ---")
    
    # context helps the LLM know what happened before (optional)
    context = "\n".join(state["results"])
    
    messages = [
        SystemMessage(content=EXECUTOR_PROMPT.format(step=current_step, context=context))
    ]
    
    response = llm.invoke(messages)
    result_text = f"Step: {current_step}\nResult: {response.content}"
    
    # Update State: Remove current step from plan, add to past_steps, add result
    return {
        "plan": plan[1:], 
        "past_steps": [current_step],
        "results": [result_text] # In LangGraph this usually appends if you use Annotated[List, operator.add]
                                 # Here we are just returning the list to be merged by the graph
    }

def reporter_node(state: PlanState):
    print("\n--- REPORTER: Summarizing all findings ---")
    
    # Combine all results into one string
    all_results = "\n\n".join(state["results"])
    
    messages = [
        SystemMessage(content=REPORTER_PROMPT.format(request=state['request'], results=all_results))
    ]
    
    response = llm.invoke(messages)
    return {"results": [response.content]} # Overwrite or append final result

# --- 4. Define the Logic (The Router) ---

def should_continue(state: PlanState):
    if len(state["plan"]) > 0:
        return "executor" # Loop back to do the next step
    else:
        return "reporter" # All steps done

# --- 5. Build the Graph ---

workflow = StateGraph(PlanState)

# Add Nodes
workflow.add_node("planner", planner_node)
workflow.add_node("executor", executor_node)
workflow.add_node("reporter", reporter_node)

# Add Edges
workflow.add_edge(START, "planner")
workflow.add_edge("planner", "executor")

# The Conditional Loop
workflow.add_conditional_edges(
    "executor",
    should_continue,
    {
        "executor": "executor",
        "reporter": "reporter"
    }
)

workflow.add_edge("reporter", END)

app = workflow.compile()

# --- 6. Run it ---
# Note: "results" uses a reducer logic in real apps, here we just simplified for clarity
# If you run this, you will see the 'results' list might behave differently depending on graph config.
# For this simple demo, we assume the state merges nicely.

print(">>> STARTING PLANNING AGENT")
inputs = {
    "request": "Research the history of the Eiffel Tower, focusing on its construction and initial reception.",
    "plan": [],
    "results": [], 
    "past_steps": []
}

# The invoke will run the loop until plan is empty
final_output = app.invoke(inputs)

print("\n\n>>> FINAL REPORT <<<")
# The last item in results is the reporter's summary
print(final_output["results"][-1])

>>> STARTING PLANNING AGENT

--- PLANNER: Analyzing request 'Research the history of the Eiffel Tower, focusing on its construction and initial reception.' ---
--- PLANNER: Created roadmap: ['Gather credible sources on the history of the Eiffel Tower', 'Analyze information on its construction process', 'Investigate the initial public and critical reception'] ---
--- EXECUTOR: Working on step: 'Gather credible sources on the history of the Eiffel Tower' ---
--- EXECUTOR: Working on step: 'Analyze information on its construction process' ---
--- EXECUTOR: Working on step: 'Investigate the initial public and critical reception' ---

--- REPORTER: Summarizing all findings ---


>>> FINAL REPORT <<<
The Eiffel Tower, an iconic symbol of Paris, has a rich history marked by controversy and eventual acceptance. Its construction began in 1887 and was completed in 1889 as the centerpiece of the 1889 Exposition Universelle, a world's fair held to celebrate the 100th anniversary of the French Revo